<a href="https://colab.research.google.com/github/najnin26/DL_Algo_Implement/blob/main/LSTM/Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re


In [6]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

In [7]:
file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

In [8]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [9]:
# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [10]:
# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [11]:
# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [12]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
					input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# Train the model
model.fit(X, y, epochs=150, verbose=1)

Epoch 1/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 90ms/step - accuracy: 0.0582 - loss: 5.5802
Epoch 2/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.0895 - loss: 5.0709
Epoch 3/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.0899 - loss: 5.0111
Epoch 4/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step - accuracy: 0.0794 - loss: 4.9868
Epoch 5/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - accuracy: 0.0780 - loss: 5.0234
Epoch 6/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.0716 - loss: 4.9965
Epoch 7/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.0885 - loss: 4.9926
Epoch 8/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.0868 - loss: 4.9860
Epoch 9/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.0847 - loss: 4.9454
Epoch 10/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 132ms/step - accuracy: 0.0956 - loss: 4.8757
Epoch 11/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.0715 - loss: 4.9401
Epoch 12/150
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/s

In [14]:
# Generate next word predictions
seed_text = "what is the fee"
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Next predicted words: what is the fee of monthly subscription fee for


In [15]:
import time
text = "what is the fee"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
what is the fee of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
what is the fee of monthly
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
what is the fee of monthly subscription
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
what is the fee of monthly subscription fee
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
what is the fee of monthly subscription fee for
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
what is the fee of monthly subscription fee for data
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
what is the fee of monthly subscription fee for data science
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
what is the fee of monthly subscription fee for data science sessions
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
what is the fee of monthly subscription fee for data science sessions in
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
what is the fee of monthly subscription fee for data science sessions in the
